1. PostgreSQL query


In [1]:
import psycopg2
from psycopg2.extras import DictCursor
from prettytable import PrettyTable
from datetime import datetime

connection = psycopg2.connect(
    database="thingsboard",
    user="thingsboard",
    password="postgres",
    host="localhost",
    port=5432,
)

print("Successfully connected to the database.")

cursor = connection.cursor(cursor_factory=DictCursor)

query = """
SELECT uuid, ts, key
FROM ts_kv
WHERE entity_id = 'b4f415d0-7db2-11ee-b0f3-69bd975277c1'
LIMIT 5;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Create a PrettyTable object
table = PrettyTable()
table.field_names = ["uuid", "ts", "key"]

# Add rows to the table
for row in rows:
    # Convert the timestamp to a formatted date and time
    formatted_ts = datetime.utcfromtimestamp(row["ts"] / 1000.0).strftime(
        "%Y-%m-%d %H:%M:%S"
    )
    # Add the row with the formatted timestamp
    table.add_row([row["uuid"], formatted_ts, row["key"]])

# Print the table
print(table)

# Close the cursor and the database connection
cursor.close()
connection.close()

Successfully connected to the database.
+--------------------------------------+---------------------+-----+
|                 uuid                 |          ts         | key |
+--------------------------------------+---------------------+-----+
| 54f310d5-f091-4c33-b703-543c39955840 | 2023-11-15 20:38:35 |  73 |
| 19194c07-00dc-431d-897b-97096fc0a971 | 2023-11-15 20:38:35 |  74 |
| 5e9d8acc-fe9c-43bd-ba61-8d5e93f9b2b5 | 2023-11-15 20:38:35 |  75 |
| 3aaffcc5-4fa7-477b-b7dd-935873f0c2be | 2023-11-15 20:38:35 |  76 |
| 8d499c66-c092-4e66-a6a1-41d0b718327c | 2023-11-15 20:38:35 |  77 |
+--------------------------------------+---------------------+-----+


C:\Users\c21054458\AppData\Local\Temp\ipykernel_16004\3634965137.py:35: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  formatted_ts = datetime.utcfromtimestamp(row["ts"] / 1000.0).strftime(


2. SPARQL query


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import csv

endpoint_url = "http://localhost:3030/abacws-sensor-network/sparql"

sparql_query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX ref: <http://example.org/reference#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?sensor ?location
WHERE {
  VALUES ?uuid {
    "54f310d5-f091-4c33-b703-543c39955840"
    "19194c07-00dc-431d-897b-97096fc0a971"
    "5e9d8acc-fe9c-43bd-ba61-8d5e93f9b2b5"
    "3aaffcc5-4fa7-477b-b7dd-935873f0c2be"
    "8d499c66-c092-4e66-a6a1-41d0b718327c"
  }

  ?sensor rdf:type brick:Sensor .
  ?sensor brick:hasLocation ?location .
}
        """


def execute_sparql_query(sparql_query, endpoint_url):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat("json")
    results = sparql.query().convert()
    return results


ans = execute_sparql_query(sparql_query, endpoint_url)

# Print headers
headers = ans["head"]["vars"]
print("Headers:", headers)

# Print data rows
data_rows = []
for binding in ans["results"]["bindings"]:
    row = [binding.get(var, {}).get("value", "") for var in headers]
    data_rows.append(row)

print("Data Rows:")
for row in data_rows:
    print(row)

Headers: ['sensor', 'location']
Data Rows:
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.01', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.01']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.02', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.02']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.03', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.03']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.04', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.04']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.05', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.05']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.06', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.06']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.07', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.07']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.08', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.08']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.09', 'http://abacw

Combining both codes


In [3]:
import psycopg2
from psycopg2.extras import DictCursor
from prettytable import PrettyTable
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON

# First Code: Retrieve UUIDs from PostgreSQL
connection = psycopg2.connect(
    database="thingsboard",
    user="thingsboard",
    password="postgres",
    host="localhost",
    port=5432,
)

print("Successfully connected to the database.")

cursor = connection.cursor(cursor_factory=DictCursor)

query = """
SELECT uuid
FROM ts_kv
WHERE entity_id = 'b4f415d0-7db2-11ee-b0f3-69bd975277c1'
LIMIT 5;
"""

cursor.execute(query)
uuid_rows = cursor.fetchall()

# Extract UUIDs from the PostgreSQL result
uuids = [str(row["uuid"]) for row in uuid_rows]

# Close the cursor and the database connection
cursor.close()
connection.close()

# Second Code: Use UUIDs in SPARQL query
endpoint_url = "http://localhost:3030/abacws-sensor-network/sparql"

sparql_query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX ref: <http://example.org/reference#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?sensor ?location
WHERE {
  VALUES ?uuid {
    "%s"
  }

  ?sensor rdf:type brick:Sensor .
  ?sensor brick:hasLocation ?location .
}
""" % '" "'.join(
    uuids
)


# Execute SPARQL query
def execute_sparql_query(sparql_query, endpoint_url):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat("json")
    results = sparql.query().convert()
    return results


ans = execute_sparql_query(sparql_query, endpoint_url)

# Print headers
headers = ans["head"]["vars"]
print("Headers:", headers)

# Print data rows
data_rows = []
for binding in ans["results"]["bindings"]:
    row = [binding.get(var, {}).get("value", "") for var in headers]
    data_rows.append(row)

print("Data Rows:")
for row in data_rows:
    print(row)

Successfully connected to the database.
Headers: ['sensor', 'location']
Data Rows:
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.01', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.01']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.02', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.02']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.03', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.03']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.04', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.04']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.05', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.05']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.06', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.06']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.07', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.07']
['http://abacwsbuilding.cardiff.ac.uk/abacws#airq5.08', 'http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.08']
['http://abacwsbuilding.cardi